In [21]:
#pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

Note: you may need to restart the kernel to use updated packages.


In [22]:
#pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [14]:
# import the required libraries 
from googleapiclient.discovery import build 
from google_auth_oauthlib.flow import InstalledAppFlow 
from google.auth.transport.requests import Request 
import pickle 
import os.path 
import base64 
import email 
from bs4 import BeautifulSoup 
import re

# Define the SCOPES. If modifying it, delete the token.pickle file. 
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly'] 

def getEmails(): 
	# Variable creds will store the user access token. 
	# If no valid token found, we will create one. 
	creds = None

	# The file token.pickle contains the user access token. 
	# Check if it exists 
	if os.path.exists('token.pickle'): 

		# Read the token from the file and store it in the variable creds 
		with open('token.pickle', 'rb') as token: 
			creds = pickle.load(token) 

	# If credentials are not available or are invalid, ask the user to log in. 
	if not creds or not creds.valid: 
		if creds and creds.expired and creds.refresh_token: 
			creds.refresh(Request()) 
		else: 
			flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES) 
			creds = flow.run_local_server(port=0) 

		# Save the access token in token.pickle file for the next run 
		with open('token.pickle', 'wb') as token: 
			pickle.dump(creds, token) 

	# Connect to the Gmail API 
	service = build('gmail', 'v1', credentials=creds) 

	# request a list of all the messages 
	result = service.users().messages().list(maxResults= 5, userId='me').execute() 

	# We can also pass maxResults to get any number of emails. Like this: 
	# result = service.users().messages().list(maxResults=200, userId='me').execute() 
	messages = result.get('messages') 

	# messages is a list of dictionaries where each dictionary contains a message id. 

	# iterate through all the messages 
	for msg in messages: 
		# Get the message from its id 
		txt = service.users().messages().get(userId='me', id=msg['id']).execute() 

		# Use try-except to avoid any Errors 
		try: 	
			# Get value of 'payload' from dictionary 'txt' 
			payload = txt['payload'] 
			headers = payload['headers'] 

			# Look for Subject and Sender Email in the headers 
			for d in headers: 
				if d['name'] == 'Subject': 
					subject = d['value'] 
				if d['name'] == 'From': 
					sender = d['value']

			email_addy = r'<(.*?)>'
			match = re.search(email_addy, sender)
			if match:
				email_address = match.group(1)
			else:
				email_address = "No email address found in the given text."				
	 		
			# The Body of the message is in Encrypted format. So, we have to decode it. 
			# Get the data and decode it with base 64 decoder. 
			parts = payload.get('parts')[0] 
			data = parts['body']['data'] 
			data = data.replace("-","+").replace("_","/") 
			decoded_data = base64.b64decode(data) 
			body = decoded_data.decode("utf-8")
			#print(decoded_data)
			#print(type(decoded_data))
			

			# Now, the data obtained is in lxml. So, we will parse 
			# it with BeautifulSoup library 
			# soup = BeautifulSoup(decoded_data , "lxml") 
			# body = soup.body() 
			# Printing the subject, sender's email and message 
			print("Subject: ", subject) 
			print("From: ", sender)
			print("Email address: ", email_address) 
			print("Message: ", body) 
			print('\n') 
		except: 
			pass


getEmails()

Subject:  Re: You are invited to Coworking at the Respell Loft
From:  Pritwik Jeeva <pritwikjeeva@eesa.ai>
Email address:  pritwikjeeva@eesa.ai
Message:  Hey thanks for the invite! I would really love to come co-work at the
Respell loft, but I unfortunately have a fully packed schedule tomorrow.
Will you guys be hosting next week? I will definitely come next time.
Thanks again and looking forward to seeing you guys!

Best regards,
Pritwik Jeeva

On Thu, Oct 19, 2023 at 3:25 PM Harpriya Bagri <noreply@luma-mail.com>
wrote:

> <https://lu.ma>
>
> You’re invited to
> *Coworking at the Respell Loft*
>
> OCT
> 20
> Friday, October 20
> 10:00 AM to 5:00 PM PDT
> 292 Ivy St ↗
> San Francisco, California
>
> <https://www.google.com/maps/search/?api=1&query=292%20Ivy%20St&query_place_id=ChIJEZQSk5iAhYARZKpgZof2ppw>
>
> ​Another week, another coworking sesh :)
> Accept Invite <https://lu.ma/i/c2J9TmAtNP>
>
> Can't make it? Decline the invite <https://lu.ma/i/c2J9TmAtNP> to stop
> receiving email